In [ ]:
#default_exp vision.gan.loss

In [ ]:
#hide
from IPython.display import clear_output
from nbdev.export import notebook2script
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
import torch
import logging
import numpy as np 
import pandas as pd
from torch import nn
import altair as alt
import torch.nn.functional as F
from functools import partial


logger = logging.getLogger()
logger.setLevel("INFO")

# vision.gan.loss

> 實作對抗生成網路（Generative Adversarial Network） 文獻中常見的損失函數。此模組主要提供可依照對抗類型（adversarial type） 取得生成器與辨別器對應損失函數的介面。

## Utilies

### 建立跟 logits 同 device 的 label tensor

In [ ]:
#export
def create_like(t, func, on_gpu=False):
    t2 = func(t)
    if on_gpu:
        t2 = t.cuda(t.device.index)
    return t2


def ones_like(t, on_gpu=False):
    return create_like(t, torch.ones_like, on_gpu)


def zeros_like(t, on_gpu=False):
    return create_like(t, torch.zeros_like, on_gpu)

### 依對抗類型取得相應生成器與鑑別器損失函數

In [ ]:
#export
def get_adversarial_loss_fns(type, is_logits=True, reduction="mean", on_gpu=False):
    if type == "gan":
        fn = get_gan_loss_fns
    elif type == "lsgan":
        fn = get_lsgan_loss_fns
    elif type == "wgan":
        fn = get_wgan_loss_fns
    elif type == "geometric_gan":
        fn = partial(get_hinge_loss_fns, version="geometric_gan")
    elif type == "sngan":
        fn = get_sngan_loss_fns
    elif type == "hinge_v2":
        fn = partial(get_hinge_loss_fns, version="v2")
    else:
        raise NotImplementedError
        
    return fn(is_logits=is_logits, reduction=reduction, on_gpu=on_gpu)

### 渲染不同對抗類型的損失函數

In [ ]:
#export
def draw_adversarial_loss_fns(type, arange=(-3, 3., 0.1), reduction="none", title=None):
    
    g_loss_fn, d_loss_fn = get_adversarial_loss_fns(type, reduction=reduction)
    x = torch.arange(*arange)
    real_logits = fake_logits = x.unsqueeze(-1)
    
    # generator loss
    g_loss = g_loss_fn(fake_logits)
    
    # discriminator loss
    d_real_loss, d_fake_loss = d_loss_fn(real_logits, fake_logits)
    
    # build dataframe for easy rendering
    x = x.squeeze().numpy()
    data = pd.DataFrame.from_dict(dict(
        real_logits=x,
        fake_logits=x,
        g_loss=g_loss.squeeze().numpy(),
        d_real_loss=d_real_loss.squeeze().numpy(),
        d_fake_loss=d_fake_loss.squeeze().numpy(),
    ), orient="columns")
    
    # draw loss functions
    d_real_panel = alt.Chart(data, title='[D] real loss').mark_line().encode(
        x="real_logits",
        y="d_real_loss")
    d_fake_panel = alt.Chart(data, title='[D] fake loss').mark_line().encode(
        x="fake_logits",
        y="d_fake_loss")
    g_panel = alt.Chart(data, title='[G] loss').mark_line().encode(
        x="fake_logits",
        y="g_loss"
    )

    chart = alt.hconcat(d_real_panel, d_fake_panel, title=type)
    chart &= g_panel
    
    return chart

## GAN

In [ ]:
#export
def get_gan_loss_fns(is_logits=True, reduction="mean", on_gpu=False, **kwargs):
    if is_logits:
        bce = partial(F.binary_cross_entropy_with_logits, reduction=reduction)
    else:
        bce = partial(F.binary_cross_entropy, reduction=reduction)

    def g_loss_fn(fake_logits, on_gpu=False):
        out = bce(fake_logits, ones_like(fake_logits, on_gpu))
        return out
    
    def d_loss_fn(real_logits, fake_logits, on_gpu=False):
        real_loss = bce(real_logits, ones_like(real_logits, on_gpu))
        fake_loss = bce(fake_logits, zeros_like(fake_logits, on_gpu))
        return real_loss, fake_loss
    
    return g_loss_fn, d_loss_fn

In [ ]:
#server
draw_adversarial_loss_fns("gan")

alt.VConcatChart(...)

### LSGAN

In [ ]:
#export
def get_lsgan_loss_fns(reduction="mean", on_gpu=False, **kwargs):
    mse = partial(F.mse_loss, reduction=reduction)

    def g_loss_fn(fake_logits, on_gpu=False):
        return mse(fake_logits, ones_like(fake_logits, on_gpu))
    
    def d_loss_fn(real_logits, fake_logits, on_gpu=False):
        real_loss = mse(real_logits, ones_like(real_logits, on_gpu))
        fake_loss = mse(fake_logits, zeros_like(fake_logits, on_gpu))
        return real_loss, fake_loss

    return g_loss_fn, d_loss_fn

In [ ]:
#server
draw_adversarial_loss_fns("lsgan")

alt.VConcatChart(...)

## WGAN

In [ ]:
#export
def get_wgan_loss_fns(reduction="mean", **kwargs):
    
    def g_loss_fn(fake_logits):
        fake_loss = -fake_logits
        if reduction == "mean":
            fake_loss = fake_loss.mean()
        return fake_loss
    
    def d_loss_fn(real_logits, fake_logits):
        real_loss = -real_logits
        fake_loss = fake_logits
        if reduction == "mean":
            real_loss = real_loss.mean()
            fake_loss = fake_loss.mean()
            
        return real_loss, fake_loss

    return g_loss_fn, d_loss_fn

In [ ]:
#server
draw_adversarial_loss_fns("wgan")

alt.VConcatChart(...)

## Hinge Loss related

In [ ]:
#export
def get_hinge_loss_fns(version="geometric_gan", reduction="mean", on_gpu=False, **kwargs):
    
    if version == "geometric_gan":
        def g_loss_fn(fake_logits, reduction="mean", **kwargs):
            fake_loss = -fake_logits
            if reduction == "mean":
                fake_loss = fake_loss.mean()
            return fake_loss
    else:
        def g_loss_fn(fake_logits, reduction="mean", on_gpu=False):
            fake_loss = torch.max(1 - fake_logits, zeros_like(fake_logits, on_gpu))
            if reduction == "mean":
                fake_loss = fake_loss.mean()
            return fake_loss
        
    def d_loss_fn(real_logits, fake_logits, reduction="mean", on_gpu=False):
        real_loss = torch.max(1 - real_logits, zeros_like(real_logits, on_gpu))
        fake_loss = torch.max(1 + fake_logits, zeros_like(fake_logits, on_gpu))
        if reduction == "mean":
            real_loss = real_loss.mean()
            fake_loss = fake_loss.mean()
        
        return real_loss, fake_loss

    return partial(g_loss_fn, reduction=reduction), partial(d_loss_fn, reduction=reduction)

In [ ]:
#server
draw_adversarial_loss_fns("geometric_gan")

alt.VConcatChart(...)

## SNGAN

Optimizing these objectives is equivalent tominimizing the so-called reverse KL divergence. It's a kind of hinge loss.

In [ ]:
#export
def get_sngan_loss_fns(reduction="mean", on_gpu=False, **kwargs):
    
    def g_loss_fn(fake_logits, reduction="mean", on_gpu=False):
        fake_loss = -fake_logits
        if reduction == "mean":
            fake_loss = fake_loss.mean()
        return fake_loss
        
    def d_loss_fn(real_logits, fake_logits, reduction="mean", on_gpu=False):
        real_loss = torch.min(-1 + real_logits, zeros_like(real_logits, on_gpu))
        fake_loss = torch.min(-1 - fake_logits, zeros_like(fake_logits, on_gpu))
        if reduction == "mean":
            real_loss = real_loss.mean()
            fake_loss = fake_loss.mean()
        
        return real_loss, fake_loss

    return partial(g_loss_fn, reduction=reduction), partial(d_loss_fn, reduction=reduction)

In [ ]:
#server
draw_adversarial_loss_fns("sngan")

alt.VConcatChart(...)

In [ ]:
#hide
notebook2script()
clear_output()